# LANL Earthquake Prediction

## Preparation

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
pd.options.display.float_format = '{:,.10f}'.format

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
from src.utils.file import FileUtils as file

data = file.train_data()
data.columns = ['x', 'y']
data['id'] = 1
data.head(1)

Reading file:data/train/train.csv into dataframe:


In [5]:
file.split_and_cache(data)

In [7]:
from tsfresh import extract_features

def extract_tf_features(tupl):
    segment,seg = tupl[0],tupl[1]
    print('extract_tf_features.call.segment={segment}'.format(segment=segment))
    
    x = seg[['id','x']]   # pd series
    y = seg['y'].values[-1]  # single value
    
    extracted_features = extract_features(x, column_id='id')
    
    X_train = pd.DataFrame(dtype=np.float64)
    y_train = pd.DataFrame(dtype=np.float64)
    
    X_train = X_train.append(extracted_features)
    y_train.loc[segment, 'y'] = y
    
    X_train['y'] = y
    X_train.to_csv(segments_path+str(segment))
    
    print('extract_tf_features.success.segment={segment}'.format(segment=segment))
    
    del seg
    del x
    del y
    del extracted_features
    del X_train
    del y_train

In [ ]:
import multiprocessing

num_procs = multiprocessing.cpu_count() - 2
num_procs

6

In [ ]:
from multiprocessing.dummy import Pool
pool = Pool(num_procs)

pool.map(extract_tf_features, [(segment, data.iloc[segment*SEGMENT_ROWS:segment*SEGMENT_ROWS+SEGMENT_ROWS]) for segment in range(segments)])


extract_tf_features.call.segment=0
extract_tf_features.call.segment=525
extract_tf_features.call.segment=1050
extract_tf_features.call.segment=1575
extract_tf_features.call.segment=2100
extract_tf_features.call.segment=2625


Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]

Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]


Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]



Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]




Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# segments_path = '{cache_dir}{segments_file}'.format(cache_dir=CACHE_DIR,segments_file=SEGMENTS_FILE)

# X_train['y']=y_train['y']
# X_train.to_csv(segments_path)